## Import

In [8]:
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from keras.utils import to_categorical

### ffmpeg-python 다운로드 (librosa load 에러 방지)

In [4]:
!pip install ffmpeg-python

## 소리 데이터 전처리 코드

In [11]:
def preprocessing(fir_file):
  y, sr = librosa.load(fir_file)

  # D = np.abs(librosa.stft(y, n_fft = 4096, win_length = 4096, hop_length = 2028))
  mfcc = np.mean(librosa.feature.mfcc(y, sr = sr, n_mfcc = 40).T , axis = 0)
  melspec = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
  chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
  chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
  chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
  featureset = np.reshape(np.vstack((mfcc,melspec,chroma_stft,chroma_cq,chroma_cens)),(40,5))

  # delta2_mfcc = librosa.feature.delta(mfcc, order=2)

  return featureset

In [6]:
# csv 읽어오기, 경로설정
df = pd.read_csv('/content/UrbanSound8K.csv')

df.rename(columns={'class':'label'},inplace=True)
classes=list(np.unique(df.label))

for_nums = []
path = '/content/drive/MyDrive/audio'  # 경로를 자신의 환경에 맞게 바꿔주세요.
a = 0

In [14]:
# version4 - urban8K 데이터 전처리
x_tr, x_t, y_tr, y_t = [],[],[],[]
x, xt = [], []

for i in tqdm(range(len(df))):
  a = i
  fold = df.iloc[a]['fold']
  label_name = df.iloc[a]['label']
  path_now = '/content/drive/MyDrive/audio'+'/fold'+str(fold)+'/'+df.iloc[a]['slice_file_name']  # 경로를 자신의 환경에 맞게 바꿔주세요.
  featureset_now = preprocessing(path_now)
  print(str(a)+' : '+path_now + ' fold'+str(fold)+' '+label_name)
  
  if fold != 10 :
    x_tr.append(featureset_now)
    print('---------------------------')
    print(len(x_tr[i]))  # 전처리된 list의 길이.
    print(x_tr[i])       # 전처리된 list.
    y_tr.append(classes.index(label_name))
  else: 
    x_t.append(featureset_now)
    y_t.append(classes.index(label_name))

 10%|█         | 1/10 [00:00<00:08,  1.06it/s]

0 : /content/drive/MyDrive/audio/fold5/100032-3-0-0.wav fold5 dog_bark
---------------------------
40
[-209.93579487   63.42262762 -123.69024252  -60.0192051   -14.45738451
  -29.09064261   -4.25117803   12.41974909   12.73930311    9.05396662
   -6.80807787   -2.13655875  -13.34995233    6.67802526   -3.1447943
   -3.69539251  -15.76819459   -0.38718634   -3.25079328    6.49081244
  -11.35261306   -7.68830237    5.41635442   -2.70299347   -6.27998139
    0.45257506   -3.87933872    5.36968987   -6.66416289   -4.45491797
    0.39725508    8.10058753   -1.07305265    2.7207087    -6.71397527
   -1.87979063    2.01220858   -0.662542     -1.89823113    1.35495249]


 20%|██        | 2/10 [00:01<00:07,  1.10it/s]

1 : /content/drive/MyDrive/audio/fold5/100263-2-0-117.wav fold5 children_playing
---------------------------
40
[-417.29110294  100.14850782  -43.86925184   51.95310994    9.04440675
    8.78020594   10.37592157    2.73138798    6.17525893    5.06514314
   -4.37066091   14.67556814   -3.52997652    8.04368789    4.82146799
    2.61332701    1.05057824    7.75882477   -3.13875371    4.10315077
   -0.56388695    6.83062388   -0.80798987    7.22863538   -0.79897929
    0.75396174   -4.16833958    1.25981485   -6.39623348    1.32272064
   -1.67411569    2.70545895   -1.92235785    1.1593204    -1.2141578
    1.20422766   -2.9931971     1.98513575   -2.07825577    1.9851764 ]


 30%|███       | 3/10 [00:02<00:06,  1.08it/s]

...

7 : /content/drive/MyDrive/audio/fold5/100263-2-0-3.wav fold5 children_playing
---------------------------
40
[-4.57554645e+02  1.07447551e+02 -2.00310651e+01  4.08115502e+01
  3.91386575e+00  2.29272573e+01  1.51526193e+01  8.50083501e+00
  5.60075352e+00  6.57633513e+00 -2.89469178e+00  1.00850361e+01
  7.31834196e-01  3.80646878e+00  1.44381808e+00  1.33317326e+00
 -7.12724379e-01  6.95571506e+00 -7.26870957e-01  4.53915375e+00
 -1.63217225e+00  3.57265367e+00 -1.85890791e+00  4.03586374e+00
 -4.17055439e+00  8.01586242e-01 -3.21017827e+00 -2.47542177e-01
 -2.39334192e+00  2.14404147e+00 -6.29621871e-01  9.91637584e-01
 -2.90800607e+00  3.21704387e+00 -3.64454105e+00  8.14450676e-01
 -2.19099544e+00 -8.12068690e-02 -4.12332214e+00  2.57176450e+00]


 90%|█████████ | 9/10 [00:07<00:00,  1.17it/s]

8 : /content/drive/MyDrive/audio/fold5/100263-2-0-36.wav fold5 children_playing
---------------------------
40
[-4.66591311e+02  1.18987923e+02 -3.07155156e+01  5.13060847e+01
  4.39498207e+00  2.23829967e+01  1.28650577e+01  8.00212277e+00
  7.45460902e+00  3.84689032e+00 -3.02372864e+00  1.37575048e+01
 -1.29876375e-01  8.81949543e+00  1.07389157e+00  2.82526937e+00
 -2.23166284e+00  4.05393237e+00 -3.97705986e+00  3.83645669e+00
 -8.92393952e-01  4.67080350e+00  2.45283710e-01  4.44679642e+00
 -1.28048735e+00 -9.81458446e-01 -2.18376330e+00  6.75677289e-02
 -1.72919420e+00  1.29453027e+00 -4.81198605e-01  2.80003108e+00
 -1.51640605e+00  2.17009251e+00 -2.14968316e+00  4.67492408e-01
 -3.83251119e+00  1.49996657e+00 -1.97156771e+00 -9.72801681e-01]


100%|██████████| 10/10 [00:12<00:00,  1.20s/it]

9 : /content/drive/MyDrive/audio/fold10/100648-1-0-0.wav fold10 car_horn


In [15]:
# version4 - 학습 데이터 만들기
x.append(x_tr)  

xt.append(x_t)

# numpy array로 만들기
# train용 
x = np.array(x)
y = np.array(y_tr)

# test용
xt = np.array(xt)
y_t = np.array(y_t)

# z-score normalization
x_train = (x-np.mean(x)/np.std(x))
x_test = (xt-np.mean(xt)/np.std(xt))

y_train = to_categorical(y, num_classes = 10)
y_test = to_categorical(y_t, num_classes = 10)

In [16]:
# version3~4 - numpy 배열을 .npy 파일로 저장
np.save('/content/drive/MyDrive/GSM_Festival_2020_datas/x_train', x_train)
np.save('/content/drive/MyDrive/GSM_Festival_2020_datas/x_test', x_test)
np.save('/content/drive/MyDrive/GSM_Festival_2020_datas/y_train', y_train)
np.save('/content/drive/MyDrive/GSM_Festival_2020_datas/y_test', y_test)